In [78]:
import pandas as pd
import numpy as np
from math import log
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

Считываем и переименовываем колонки с цифер на их суть для наглядности:

In [79]:
df=pd.read_table('SMSSpamCollection',header=None)
df.rename(columns = {0: 'label', 1: 'message'}, inplace = True)
df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


Замапим для удобности названия классов на цифры:

In [80]:
df['label'] = df['label'].map({'ham': 0, 'spam': 1})
df.head()

,label,message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


Сделаем предобработку текстов сообщений,как-то:


-приведем все до lowerCase

-разобьем на токены

-Пользуем двуграммы так как это добавит точности классификации,потому что 'good' и 'not good' вещи разные

-уберём служебные слова и местоимения (они только сбивать будут,их в каждом предложении полно,а смысловой нагрузки не несут) 

-пользуем стеммер Портера,чтобы вернуть слова в начальную форму(одно и то же слово в разных формах все равно одно и то же слово)

In [89]:
def preprocessMessages(message,gram=2):
    message=message.lower()
    words=word_tokenize(message)
    words = [w for w in words if len(w) > 2]
    if gram > 1:
        w = []
        for i in range(len(words) - gram + 1):
            w += [' '.join(words[i:i + gram])]
        return w
    stop_words=stopwords.words('english')
    words = [word for word in words if word not in stop_words]
    words = [PorterStemmer().stem(word) for word in words]
    return words

Итак,теперь нам нужен NB,гуглим,что есть по формулам:


1)$$P(\omega _{i}\,|\,c)=\frac{W_{ic}}{\Sigma _{{i}'\in V} W^{_{{i}'c}}} $$

Где:

 $$W{_{{i}'c}}- количество\,раз\,сколько\,i-ое\,слово\,встречается\,в\,докуметнах\,класса\,C$$

 $$V-список\,всех\,уникальных\,слов\,(словарь\,корпуса)$$

Не подойдет,не адекватно поведет себя при встрече с новым словом,значение обнулится => документ неззя класифицировать, у него нулевая вероятность по всем классам

Юзаем сглаживание Лапласса(т.е.делаем вид,что видели уже слово хотя бы раз),потому что самое очевидное решение:

2)  
$$P(\omega _{i}\,|\,c)=\frac{W_{ic}+1}{\Sigma _{{i}'\in V} (W^{_{{i}'c}}+1)}=\frac{W_{ic}+1}{|V|+\Sigma _{{i}'\in V}W^{_{{i}'c}}}$$


Пойдет,но слишком теория,какая формула по сути?

$$\log\frac{D_c}{D} + \sum_{i \in Q}\log{\frac{W_{ic}+1}{ |V|+L_{c} }}\,\,\,(1)$$

Где:


$$D_c-количество\,документов\,в\,обучающей\,выборке\,принадлежащих\,классу\, с$$

$$D-общее\,количество\,докуметов\,в\,обучающей\,выборке$$

$$|V|-количетсво\,уникальных\,слов\,во\,всех\,документах\,обучающей\,выборки$$

$$L_{c}-суммарное\,количество\,слов\,в\,документах\,класса\,с\,в\,обучающей\,выборке$$

$$W_{ic}-сколько\,раз\,i-ое\,слово\,встречалось\,в\,документах\,класса\,с\,в\,обучающей\,выборке$$

$$Q-множество\,слов\,классифицируемого\,документа\,(включая\,повторы)$$

Ну вот теперь можно действовать,сделаем две штуки,с апдейтом веса слов(по идее,это должно добавить точности алгоритму)т.е.TF-IDF и без т.е.Bag of Words:

В кратце о подходах:

1) Bag of words- модель,где мы высчитываем "term freuency" то есть количество появлений каждого слова в датасете:

$$P(\omega\,|\,Ham/Spam)=\frac{Total\,count\,of\,word\,\omega\,in\,Ham/Spam\,messages }{Total\,count\,of\,words\,in\,dataset}\,\,\,(2)$$

2) TF-IDF(то есть Term Frequency-Inverse Documnet Frequency)-в дополнении к "term frequency" высчитываем "inverse document frequency":

$$IDF(\omega)=log\frac{Total\,count\,of\,messages}{Total\,count\,of\,messages\,with\, \omega}$$

Все равно не понятно,что считать-то? Яжпрограмист,конкретнее бы......

$$P(\omega\,|\,Ham/Spam)=\frac{TF(\omega\,|\,Ham/Spam)*IDF(\omega)}{\sum_{\forall\,words\,x\, \in \,train\,dataset } TF(x\,|\,Ham/Spam)*IDF(x)}\,\,\,(3)$$

Теперь можно и запрогать:

In [90]:
class NBclassifier(object):
    
    def __init__(self, trainData,approach):
        self.messages,self.label=trainData['message'],trainData['label']
        self.approach=approach
        
    def fit(self):
        #Высчитываем необходимые велечины:
        self.neededDataForFormulas()
        #Применяем выбранный метод:
        if self.approach == 'Bag of Words':
            self.BOW()
        if self.approach== 'TF-IDF':
            self.TF_IDF()
 

    def neededDataForFormulas(self):
        # Какое кол-во строк в DF:
        countOfMessages=self.messages.shape[0]
        #Кол-во Ham и Spam строк в DF:
        self.countOfHamMessage=self.label.value_counts()[0]
        self.countOfSpamMessage=self.label.value_counts()[1]
        self.totalMessage=self.countOfHamMessage+self.countOfSpamMessage
        #Кол-во Ham и Spam слов в DF:
        self.spamWords=0
        self.hamWords=0
        #Частота встречи слов из Ham и Spam:
        self.tfSpam=dict()
        self.tfHam=dict()
        self.idfSpam=dict()
        self.idfHam=dict()
        
        for i in range(countOfMessages):
            preprocessedMessage=preprocessMessages(self.messages[i])
            #Лист для отслеживания,встречалось слово или нет:
            cachedWords=list()
            
            for word in preprocessedMessage:
                #Суммарное кол-во слов в сообщениях класса Ham/Spam:
                if self.label[i]:
                    self.tfSpam[word]=self.tfSpam.get(word,0) + 1
                    self.spamWords +=1
                else:
                    self.tfHam[word]=self.tfHam.get(word,0) + 1
                    self.hamWords +=1
                # Кол-во уникальных слов во всех сообщениях:
                if word not in cachedWords:
                    cachedWords +=[word]
                #Сколько раз i-тое слово встречалось в сообщениях Ham/Spam:
            for word in cachedWords:
                if self.label[i]:
                    self.idfSpam[word]=self.idfSpam.get(word,0)+1
                else:
                     self.idfHam[word]=self.idfHam.get(word,0)+1
                        
    def BOW(self):
        #Используем формулу (2),для вычисления вероятности слова:
        self.probOfSpam = dict()
        self.probOfHam = dict()
        for word in self.tfSpam:
            #Нормированная вероятность со сглаживанием Лапласса:
            self.probOfSpam[word] = (self.tfSpam[word] + 1) / (self.countOfSpamMessage +len(list(self.tfSpam.keys())))
        for word in self.tfHam:
            #Нормированная вероятность со сглаживанием Лапласса:
            self.probOfHam[word] = (self.tfHam[word] + 1) / (self.countOfHamMessage + len(list(self.tfHam.keys())))
        #Вероятность встретить Spam/Ham сообщение:
        self.probOfSpamMessage=self.countOfSpamMessage / self.totalMessage
        self.probOfHamMessage =self.countOfHamMessage / self.totalMessage 
        
        
    def TF_IDF(self):
        self.probOfSpam=dict()
        self.probOfHam=dict()
        self.summTF_IDFforSpam=0
        self.summTF_IDFforHam=0
        #Используем формулу (3),для вычисления вероятности слова:
        for word in self.tfSpam:
            #Числитель:
            self.probOfSpam[word]=(self.tfSpam[word])*log((self.countOfSpamMessage+self.countOfHamMessage)/(self.idfSpam[word]+self.idfHam.get(word,0)))
            #Знаменатель:
            self.summTF_IDFforSpam +=self.probOfSpam[word]
        for word in self.tfSpam:
            #Нормированная вероятность со сглаживанием Лапласса:
            self.probOfSpam[word]=(self.probOfSpam[word]+1)/(self.summTF_IDFforSpam+len(list(self.probOfSpam.keys())))
            
        for word in self.tfHam:
            #Числитель:
            self.probOfHam[word]=(self.tfHam[word])*log((self.countOfSpamMessage+self.countOfHamMessage)/(self.idfHam[word]+self.idfSpam.get(word,0)))
            #Знаменатель:
            self.summTF_IDFforHam +=self.probOfHam[word]
            #Нормированная вероятность со сглаживанием Лапласса:
        for word in self.tfHam:
            self.probOfHam[word]=(self.probOfHam[word]+1)/(self.summTF_IDFforHam+len(list(self.probOfHam.keys())))
        #Вероятность встретить Spam/Ham сообщение:
        self.probOfSpamMessage=self.countOfSpamMessage/self.totalMessage
        self.probOfHamMessage=self.countOfHamMessage/self.totalMessage
        
        
    def classify(self,preprocessedMessage):
        pSpam=0
        pHam=0
        if (self.approach=='Bag of Words'):
            for word in preprocessedMessage:
                if word in self.probOfSpam:
                    #Отдельный от суммы log в формуле (1)
                    pSpam+=log(self.probOfSpam[word])
                else:
                    pSpam-=log(self.spamWords+ len(list(self.probOfSpam.keys())))
                if word in self.probOfHam:
                    #Отдельный от суммы log в формуле (1)
                    pHam+=log(self.probOfHam[word])
                else:
                    pHam-=log(self.hamWords+ len(list(self.probOfHam.keys())))
                #Сумма log-мов из формулы (1):
                pSpam+=log(self.probOfSpamMessage)
                pHam+=log(self.probOfHamMessage)
            return pSpam>=pHam
        
        if (self.approach=='TF-IDF'):
            for word in preprocessedMessage:
                if word in self.probOfSpam:
                    #Отдельный от суммы log в формуле (1)
                    pSpam+=log(self.probOfSpam[word])
                else:
                    pSpam-=log(self.summTF_IDFforSpam+len(list(self.probOfSpam.keys())))
                if word in self.probOfHam:
                    #Отдельный от суммы log в формуле (1)
                    pHam+=log(self.probOfHam[word])
                else:
                    pHam-=log(self.summTF_IDFforHam+len(list(self.probOfHam.keys())))
                #Сумма log-мов из формулы (1):
                pSpam+=log(self.probOfSpamMessage)
                pHam+=log(self.probOfHamMessage)
            return pSpam>=pHam
        
    def predict(self,testData):
        result=dict()
        for (i,message) in enumerate(testData):
            preprocessedMessages=preprocessMessages(message)
            result[i]=int(self.classify(preprocessedMessages))
        return result

Используем для оценки качества банальную Accuracy и Roc Auc Score ибо она может объективно оценить качество модели в условии несбалансированности класов,а у нас сильная несбалансированность

In [91]:
def accuracy(labels, predictions):
    truePos, trueNeg, falsePos, falseNeg = 0, 0, 0, 0
    for i in range(len(labels)):
        truePos += int(labels[i] == 1 and predictions[i] == 1)
        trueNeg += int(labels[i] == 0 and predictions[i] == 0)
        falsePos += int(labels[i] == 0 and predictions[i] == 1)
        falseNeg += int(labels[i] == 1 and predictions[i] == 0)
    return ((truePos + trueNeg) / (truePos + trueNeg + falsePos + falseNeg))

Делаем Крос-Валидацию как было сказано и обучаем,попутно фиксируя всю точность,что получается:

In [93]:
kf = KFold(n_splits=5)
for train_index, test_index in kf.split(df):
        print("TRAIN:", train_index, "TEST:", test_index)
        train_index,test_index=list(train_index),list(test_index)
        trainData = df.loc[train_index]
        trainData.reset_index(inplace=True)
        trainData.drop(['index'],axis=1,inplace=True)
        testData = df.loc[test_index]
        testData.reset_index(inplace=True)
        testData.drop(['index'],axis=1,inplace=True)
        
        model=NBclassifier(trainData,'Bag of Words')
        model.fit()
        result=model.predict(testData['message'])
        print("Accuracy Bag of Words:",accuracy(testData['label'],np.array(list(result.values()))),",ROC AUC bag of Words:",roc_auc_score(testData['label'],np.array(list(result.values()))))
    
        model=NBclassifier(trainData,'TF-IDF')
        model.fit()
        result=model.predict(testData['message'])
        print("Accuracy TF-IDF:",accuracy(testData['label'],np.array(list(result.values()))),",ROC AUC TF-IDF:",roc_auc_score(testData['label'],np.array(list(result.values()))))
        print()

TRAIN: [1115 1116 1117 ... 5569 5570 5571] TEST: [   0    1    2 ... 1112 1113 1114]
Accuracy Bag of Words: 0.9479820627802691 ,ROC AUC bag of Words: 0.8494148187258008
Accuracy TF-IDF: 0.9479820627802691 ,ROC AUC TF-IDF: 0.8518630260974507

TRAIN: [   0    1    2 ... 5569 5570 5571] TEST: [1115 1116 1117 ... 2227 2228 2229]
Accuracy Bag of Words: 0.9479820627802691 ,ROC AUC bag of Words: 0.8276865160848733
Accuracy TF-IDF: 0.9497757847533632 ,ROC AUC TF-IDF: 0.8347787146664337

TRAIN: [   0    1    2 ... 5569 5570 5571] TEST: [2230 2231 2232 ... 3341 3342 3343]
Accuracy Bag of Words: 0.9425493716337523 ,ROC AUC bag of Words: 0.8166291866207442
Accuracy TF-IDF: 0.9470377019748654 ,ROC AUC TF-IDF: 0.8317394974934441

TRAIN: [   0    1    2 ... 5569 5570 5571] TEST: [3344 3345 3346 ... 4455 4456 4457]
Accuracy Bag of Words: 0.9434470377019749 ,ROC AUC bag of Words: 0.8329586210588299
Accuracy TF-IDF: 0.9452423698384201 ,ROC AUC TF-IDF: 0.8393688774690862

TRAIN: [   0    1    2 ... 4455 

Теперь надо сравнить с коробкой в sklearn:

In [6]:
clf = MultinomialNB()
clf=Pipeline([('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [52]:
kf = KFold(n_splits=5)
for train_index, test_index in kf.split(df):
        print("TRAIN:", train_index, "TEST:", test_index)
        train_index,test_index=list(train_index),list(test_index)
        trainData = df.loc[train_index]
        trainData.reset_index(inplace=True)
        trainData.drop(['index'],axis=1,inplace=True)
        testData = df.loc[test_index]
        testData.reset_index(inplace=True)
        testData.drop(['index'],axis=1,inplace=True)
        
        clf.fit(trainData['message'],trainData['label'])
        result=clf.predict(testData['message'])
        print("Accuracy TF-IDF:",accuracy(testData['label'],result),"ROC_AUC TF_IDF:",roc_auc_score(testData['label'],result))
        print()

TRAIN: [1115 1116 1117 ... 5569 5570 5571] TEST: [   0    1    2 ... 1112 1113 1114]
Accuracy TF-IDF: 0.9560538116591928 ROC_AUC TF_IDF: 0.8541666666666667

TRAIN: [   0    1    2 ... 5569 5570 5571] TEST: [1115 1116 1117 ... 2227 2228 2229]
Accuracy TF-IDF: 0.9596412556053812 ROC_AUC TF_IDF: 0.8404255319148937

TRAIN: [   0    1    2 ... 5569 5570 5571] TEST: [2230 2231 2232 ... 3341 3342 3343]
Accuracy TF-IDF: 0.9605026929982047 ROC_AUC TF_IDF: 0.8394160583941606

TRAIN: [   0    1    2 ... 5569 5570 5571] TEST: [3344 3345 3346 ... 4455 4456 4457]
Accuracy TF-IDF: 0.9515260323159784 ROC_AUC TF_IDF: 0.8269230769230769

TRAIN: [   0    1    2 ... 4455 4456 4457] TEST: [4458 4459 4460 ... 5569 5570 5571]
Accuracy TF-IDF: 0.9614003590664273 ROC_AUC TF_IDF: 0.8517241379310345



# Вывод:

Нормально получилось,правда медленно,но это уже совсем другая история....